# 研究課題：画像キャプション
## 目標：
下記のような画像をもとに、"a dog is sitting on the grass" のようなキャプションを生成することをめざします。

![image.png](attachment:image.png)
## プロセス：
- １、データセット準備（06/23）
- ２、InceptionV3使用（06/23）
- ３、前処理（06/23）
- ４、モデル構成（09/01）
- ５、訓練（11/17）
- ６、キャプション（11/17）

In [ ]:
import tensorflow as tf
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import time
import IPython.display as display
from tensorflow.keras.preprocessing import text
from matplotlib import pyplot as plt
from google.colab import drive

# 第１章、データセット準備
- MS-COCOダンロード（イメージ、キャプション）
- ベクトルに格納
- データサイズを制限

### MS-COCO データセット
a.解説

バージョンはいくつかありますが、本研究は「train2014」を使用しています。
（Caution: 巨大ファイルのダウンロードあり、訓練用のデータセットは、13GBのファイルです。）

b.用途：

- 物体検出／セグメンテーション
- キーポイント検出／姿勢推定
- キャプション作成：画像ごとに5つのキャプション（＝自然言語による説明文）

c.Link:
https://cocodataset.org/#download

In [ ]:
drive.mount('/content/drive')
HOME_PATH = '/content/drive/My Drive/ml_datasets/'

Mounted at /content/drive


In [ ]:
## １−１、annotation(キャプション)をダウンロード
#annotation_zip = tf.keras.utils.get_file('captions.zip',
#                        cache_subdir=HOME_PATH,  ## cacheの保存パス
#                        origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
#                        extract = True)

## イメージのファイルサイズは巨大なので、直接URLからダウンロードしました。
## URL：http://images.cocodataset.org/zips/train2014.zip


PATH = HOME_PATH + 'train2014/'

In [ ]:
#count = 0
#for file in os.listdir(PATH):
#  count = count +1
#  if file == 'COCO_train2014_000000318556.jpg':
#    print(file)

#print(count)

In [ ]:
files = os.listdir(PATH)
image_list = []
for file in files:
  if file.endswith('jpg'):
    iti = file.index(".")
    if file[iti-12:iti].endswith("(1)"):
      image_id = int(file[iti-12:iti][:-3])
    else:
      image_id = int(file[iti-12:iti])
    image_list.append(image_id)

OSError: ignored

In [ ]:
## ダウンロードしたannotation(キャプション)をディスプレー
annotation_file = HOME_PATH + 'annotations/captions_train2014.json'

with open(annotation_file, 'r') as f:
    annotations = json.load(f)

annotations['annotations'][:3]

[{'image_id': 318556,
  'id': 48,
  'caption': 'A very clean and well decorated empty bathroom'},
 {'image_id': 116100,
  'id': 67,
  'caption': 'A panoramic view of a kitchen and all of its appliances.'},
 {'image_id': 318556,
  'id': 126,
  'caption': 'A blue and white bathroom with butterfly themed wall tiles.'}]

In [ ]:
## １−２、「captions」と「img_name」のpathをそれぞれのベクトルに格納
all_captions = []
all_img_name_vector = []

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

print("captions lenhth:", len(all_captions))
print("img_name lenhth:", len(all_img_name_vector))

captions lenhth: 414113
img_name lenhth: 414113


In [ ]:
all_img_name_vector[:10]

['/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000318556.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000116100.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000318556.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000116100.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000379340.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000379340.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000318556.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000318556.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000134754.jpg',
 '/content/drive/My Drive/ml_datasets/train2014/COCO_train2014_000000538480.jpg']

In [ ]:
all_captions[:10]

['<start> A very clean and well decorated empty bathroom <end>',
 '<start> A panoramic view of a kitchen and all of its appliances. <end>',
 '<start> A blue and white bathroom with butterfly themed wall tiles. <end>',
 '<start> A panoramic photo of a kitchen and dining room <end>',
 '<start> A graffiti-ed stop sign across the street from a red car  <end>',
 '<start> A vandalized stop sign and a red beetle on the road <end>',
 '<start> A bathroom with a border of butterflies and blue paint on the walls above it. <end>',
 '<start> An angled view of a beautifully decorated bathroom. <end>',
 '<start> The two people are walking down the beach. <end>',
 '<start> A sink and a toilet inside a small bathroom. <end>']

In [ ]:
# １−３、キャプションのデータサイズを制限

# captions と image_names を一緒にシャッフル
train_captions, img_name_vector = shuffle(all_captions, all_img_name_vector, random_state=666)

# シャッフルしたデータセットから最初の 10,000 のキャプションを選択
num_examples = 100000
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

# 第二章、InceptionV3使用

- InceptionV3の重みをロード
- tf.keras モデルを作成
- 特徴量のキャッシング

### Inception v3とは

Inception v3 は、画像認識モデルで、ImageNet データセットで 78.1% を超える精度を達成することがわかっています。

このモデル自体は、畳み込み層、平均プーリング層、最大プーリング層、連結層、ドロップアウト層、全結合層など、対称と非対称の構成要素で構成されています。バッチ正規化はモデル全体で広く使用され、活性化入力に適用されます。損失は Softmax を使用して計算されます。

このモデルには、深さが 48 層の畳み込みニューラル ネットワーク、入力のshapeは299x299x3、出力のshapeは8x8x2048です。

Link：https://cloud.google.com/tpu/docs/inception-v3-advanced?hl=ja

In [ ]:
# ２−１、InceptionV3 を初期化し、 Imagenet で学習済みの重みをロードする
# このレイヤーの出力の shape は8x8x2048 です

# include_top=Trueの場合  whether to include the 3 fully-connected layers at the top of the network.
image_model_top = tf.keras.applications.InceptionV3(include_top=True, weights='imagenet')
image_model_top.summary()

96112376/96112376 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)           

In [ ]:
# include_top=Falseの場合
image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
# ２−２、InceptionV3 アーキテクチャの最後の畳み込み層である tf.keras モデルを作成
new_input = image_model.input
hidden_layer = image_model.output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

### 特徴抽出とは

特徴抽出は、ネットワーク全体の学習に時間や手間をかけずに深層学習の能力を活用できる簡単で高速な方法です。これが必要とするものは学習イメージを一巡する 1 つのパスのみであるため、GPU がない場合、特に便利です。

事前学習済みのネットワークから学習済みのイメージの特徴を抽出する。

使用するデータが元のデータと非常によく似ている場合は、ネットワークの深い部分で抽出された、より具体的な特徴が役に立つ可能性が高くなります。

Link:https://jp.mathworks.com/help/deeplearning/ug/pretrained-convolutional-neural-networks.html;jsessionid=c13587b7904b8d5401f3455cd2d1

In [ ]:
## ２−３、InceptionV3 から抽出した特徴量のキャッシング

## まずは画像を InceptionV3 が期待するフォーマットに変換します。

## 画像を 299 ピクセル × 299 ピクセルにリサイズ
## preprocess_input メソッドをつかって、画像を InceptionV3 の訓練用画像のフォーマットに合致した−1 から 1 の範囲のピクセルを持つ形式に標準化しする

def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
# 期待するフォーマットのサンプルをプリント
img, image_path = load_image(all_img_name_vector[0])
print(img.shape)

UnknownError: ignored

In [ ]:
# load_imageを使って、全部画像を1 枚ずつフォマットに変換する
# 変換した特徴量を抽出する

# 重複のない画像を取得
encode_train = sorted(set(img_name_vector))

image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

# 提取文件夹内所有文件的后缀
Files=os.listdir(PATH)
for k in range(len(Files)):
    Files[k]=os.path.splitext(Files[k])[1]



In [ ]:
# 画像を 1 枚ずつネットワークに送り込み、処理結果のベクトルをディクショナリdictionaryに保管します
# (image_name --> feature_vector)
if ('.npy') not in set(Files):
    for img, path in image_dataset:
        batch_features = image_features_extract_model(img)
        batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

        for bf, p in zip(batch_features, path):
            path_of_feature = p.numpy().decode("utf-8")
            # 拡張子「.npy」データを生成するので、実行時間がかかります
            np.save(path_of_feature, bf.numpy())

# 第三章、前処理
- トークン化
- パディング
- 訓練用とテスト用を分割
- tf.dataデータセット作成

#### トークナイザとは
自然言語処理を行うためには、まず文章を単語（＝トークン）に分解するトークナイズを行い、コンピューターが処理できる入力形式に変換しなければいけません。それを実行するプログラムが「トークナイザ」です。

texts_before = ["I have a pen.", "I have an apple", "You have pen and apple."]

texts_after = [[2, 1, 5, 3], [2, 1, 6, 4], [7, 1, 3, 8, 4]]

In [ ]:
# ３−１、トークナイザ化

# Tokenizerをインスタンス化し、ボキャブラリ(Vocabulary)中のトップ 5000 語を選択
top_k = 5000
tokenizer = text.Tokenizer(num_words=top_k,
                           oov_token="<unk>",
                           filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')

# index=0の単語を追加
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

# トークン化したベクトルを生成
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [ ]:
# トークナイザ前、トークナイザ後のannotationサンプルをプリント
print("トークナイザ前：", train_captions[0])
print("トークナイザ後：", train_seqs[0])
print("1235 → ", tokenizer.index_word.get(1235))

トークナイザ前： <start> A rusted parking meter sitting next to a parking lane. <end>
トークナイザ後： [3, 2, 1461, 197, 493, 14, 21, 13, 2, 197, 1260, 4]
1235 →  meadow


### パディング
自然言語処理では入力される文はさまざまな長さになります。しかし，大量のデータを扱うためにはデータの長さがそろっていた方が効率よく処理できます。

In [ ]:
# ３−２、パティング化

# データセット中の一番長いキャプションの長さを検出
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

# アテンションの重みを格納するために使われる max_length を計算
max_length = calc_max_length(train_seqs)

print(max_length)

52


In [ ]:
# キャプションの最大長に各ベクトルをパディング
# max_length を指定しない場合、pad_sequences は自動的に計算
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

cap_vector[:2]

array([[   3,    2, 1461,  197,  493,   14,   21,   13,    2,  197, 1260,
           4,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [   3,    2,   61,   26,  986,    2,  299,  115,  279,   10,  184,
           4,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [ ]:
# ３−３、訓練用セットと検証用セットを 80-20 に分割して生成
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=666)

len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

(80000, 80000, 20000, 20000)

In [ ]:
# npyファイルをロード(imageのキャシング)
def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  return img_tensor, cap

### BATCH_SIZE，BUFFER_SIZE

注意すべきことがいくつかあります。

- 順番が重要です。
.batch の後に .shuffle すると、バッチの順番がシャッフルされますが、要素がバッチを越えてシャッフルされることはありません。
- 完全なシャッフルのため、 buffer_size をデータセットとおなじサイズに設定しています。データセットのサイズ未満の場合、値が大きいほど良くランダム化されますが、より多くのメモリーを使用します。
- シャッフルバッファがいっぱいになってから要素が取り出されます。そのため、大きな buffer_size が Dataset を使い始める際の遅延の原因になります。

Link:https://www.tensorflow.org/tutorials/load_data/images?hl=ja


In [ ]:
# ３−４、tf.dataデータセット作成（image,annotation）

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# numpyファイルを並列に読み込むためにmapを使用
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]))

# シャッフルとバッチ化
#dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.batch(BATCH_SIZE)

# prefetchを使うことで、モデルの訓練中にバックグラウンドでデータセットがバッチを取得できます。
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
# 訓練したデータから、IMAGEとTARGETを取得する
for (batch, (img_tensor, target)) in enumerate(dataset):
    if batch == 1:
        x = img_tensor
        y = target

UnknownError: ignored

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
y[0]

# 第四章、モデル構成

In [ ]:
embedding_dim = 256
units = 512
vocab_size = 5000

全結合層を

(64, 64, 2048)　→ (64, 64, 256)

In [ ]:
class CNN_Encoder(tf.keras.Model):
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        self.fc = tf.keras.layers.Dense(units=embedding_dim)

    def call(self, x):
        # x.shape:(batch_size, 64, embedding_dim)
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
# エンデーコーダーのテスト
encoder = CNN_Encoder(256)
x_encoder = encoder(x)
print(x_encoder.shape)

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):

        # features.shape: (batch_size, 64, embedding_dim)
        # hidden.shape  : (batch_size, units) →　(batch_size, 1, units)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # attention_hidden_layer.shape: (batch_size, 64, units)
        # W1(features)：(batch_size, 64, embedding_dim) → (batch_size, 64, units)
        # W2(hidden_with_time_axis)：( , 1, units) → (batch_size, 1, units)
        # (batch_size, 64, units) + (batch_size, 1, units) = (batch_size, 64, units)
        attention_hidden_layer = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        # score.shape: (batch_size, 64, units) → (batch_size, 64, 1)
        score = self.V(attention_hidden_layer)

        # attention_weight.shape: (batch_size, 64, 1)
        attention_weight = tf.nn.softmax(score, axis=1)

        # context_vector.shape:
        # (batch_size, 64, 1) * (batch_size, 64, embedding_dim) = (batch_size, 64, embedding_dim)
        context_vector = attention_weight * features

        # context_vector.shape: (batch_size, embedding_dim)
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weight

In [ ]:
# アテンションのテスト
bahdanau = BahdanauAttention(units)
hidden = tf.zeros((64, units))
context_vector, attention_weight = bahdanau(x_encoder, hidden)

print(context_vector.shape)
print(attention_weight.shape)

In [ ]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super().__init__()

        self.units = units
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size,
                                                   output_dim=embedding_dim)

        self.gru = tf.keras.layers.GRU(units=units,
                                       return_sequences=True,
                                       return_state=True)

        self.fc1 = tf.keras.layers.Dense(units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(units)

    def call(self, dec_input, features, hidden):

        # x.shape:(batch_size, 1)

        # context_vector.shape: (batch_size, embedding_dim)
        # attention_weights.shape:(batch_size, 64, 1)
        context_vector, attention_weights = self.attention(features, hidden)

        # x.shape:(batch_size, 1, embedding_dim)
        predict = self.embedding(dec_input)

        # context_vector.shape: (batch_size, embedding_dim) → (batch_size, 1, embedding_dim)
        predict = tf.concat([predict, tf.expand_dims(context_vector,axis=1)], axis=-1)

        # whole_sequence_output.shape: (batch_size, 1, units)
        # state.shape: (batch_size, units)
        whole_sequence_output, state = self.gru(predict)

        # gru: (64, 1, 512)
        # fc1: (64, 1, 512)

        predict = self.fc1(whole_sequence_output)


        predict = tf.reshape(predict, (-1, predict.shape[2]))
        # output shape == (batch_size * max_length, vocab)
        predict = self.fc2(predict)

        return predict, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [ ]:
dec_input = tf.expand_dims([0] * 64, 1)
dec_input.shape

In [ ]:
decoder = RNN_Decoder(embedding_dim, units, vocab_size)
x, state, attention_weights = decoder(dec_input, x_encoder, hidden)

In [ ]:
print(x.shape)
print(state.shape)
print(attention_weights.shape)

## チェックポイント

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
    print("REAL:",real)
    print("pred:",pred)

    mask = tf.math.logical_not(tf.math.equal(real, 0))
    print("mask11:", mask)

    loss = loss_object(real, pred)
    print("loss11:", loss)

    mask = tf.cast(mask, dtype=loss.dtype)
    print("mask22:", mask)
    loss *= mask

    print("loss22:", loss)

    return tf.reduce_sum(loss)

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(checkpoint=ckpt,
                                          directory=checkpoint_path,
                                          max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    print(ckpt_manager.latest_checkpoint)
    # restoring the latest checkpoint in checkpoint_path
    # ckpt.restore(ckpt_manager.latest_checkpoint)

# 第五章、訓練

In [ ]:
loss_plot = []

In [ ]:
@tf.function()
def train_step(image_tensor, target):
    loss = 0

    hidden = decoder.reset_state(64)
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    print("-----dec_input.shape:", dec_input.shape)

    with tf.GradientTape() as tape:
        feature = encoder(image_tensor)
        # (64, 40)
        print("target:", target.shape)

        for i in range(1, target.shape[1]):
            print("------------i:---------------", i)
            predictions, hidden, attention_weights = decoder(dec_input, feature, hidden)

            # shape=(64, 5000)
            print(i, "_predictions:", predictions)
            # shape=(64, 512)
            print(i, "_hidden:", hidden)
            # shape=(64,)
            print(i, "_attention_weights:", attention_weights)

            loss += loss_function(target[:,i], predictions)
            dec_input = tf.expand_dims(target[:,i], 1)

    total_loss = (loss / int(target.shape[1]))
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    #gradients = tf.distribute.get_replica_context().all_reduce('sum', gradients)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

In [ ]:
EPOCHS = 5

for epoch in range(start_epoch, EPOCHS):
    start_loss = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):

        loss, t_loss = train_step(img_tensor, target)

        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = loss.numpy() / int(target.shape[1])
            print(f'Epoch {epoch+1}')

    end_loss= time.time()
    print(f'Epoch {epoch+1}', end_loss-start_loss)

    loss_plot.append(total_loss/10000)
    if epoch % 5 == 0:
        ckpt_manager.save()



In [ ]:
plt.plot(loss_plot)

# 第六章、キャプション

In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, 64))
    hidden = decoder.reset_state(1)
    #hidden.shape:(1, 512)
    print("hidden.shape", hidden.shape)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    # temp_input.shape:(1, 299, 299, 3)
    print("temp_input.shape", temp_input.shape)

    image_tensor_val = image_features_extract_model(temp_input)
    #img_tensor_val.shape:(1, 8, 8, 2048)
    print("image_tensor_val.shape", image_tensor_val.shape)

    image_tensor_val = tf.reshape(image_tensor_val, (1, -1, 2048))
    #img_tensor_val.shape:(1, 64, 2048)
    print("image_tensor_val.shape", image_tensor_val.shape)

    features = encoder(image_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    result = []
    #result.append('<start>')

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
        #predictions.shape (1, 5000)
        print("predictions.shape", predictions.shape)
        print("predictions.shape", predictions.numpy())
        print("attention_weights.shape", attention_weights.shape)
        print("attention_weights222.shape", tf.reshape(attention_weights, (-1, )).shape)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()
        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()

        predicted_word = tf.compat.as_text(tokenizer.index_word[predicted_id])

        result.append(predicted_word)

        if predicted_word == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(20, 20))

    len_result = len(result)
    for i in range(len_result):
        temp_att = np.resize(attention_plot[i], (8, 8))
        grid_size = max(int(np.ceil(len_result/2)), 2)
        ax = fig.add_subplot(grid_size, grid_size, i+1)
        ax.set_title(result[i])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [ ]:
img_name_val[888]

In [ ]:
display.display(display.Image(img_name_val[888]))

In [ ]:
cap_val[888]

In [ ]:
from PIL import Image
# captions on the validation set
#rid = np.random.randint(0, len(img_name_val))
rid = 888
print("rid:", rid)
image = img_name_val[rid]
real_caption = ' '.join([tf.compat.as_text(tokenizer.index_word[i])
                         for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)
print("attention_plot:", attention_plot.shape)

print('Real Caption:', real_caption)
print('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)

In [ ]:
image_path = "/Users/xujian/Desktop/test06.jpg"
result, attention_plot = evaluate(image_path)
print('Prediction Caption:', ' '.join(result))